In [1]:
%pip install gurobipy
%pip install psutil
import os
import csv
import ast
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from gurobipy import Model, GRB, quicksum
import psutil

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import resource

def get_memory_usage_in_kb():
    process = psutil.Process()
    mem_info = process.memory_info()
    return mem_info.rss // 1024  # Convert bytes to KB

In [3]:
import resource

def set_memory_limit_gb(gb_limit):
    soft, hard = gb_limit * 1024 ** 3, gb_limit * 1024 ** 3  # Convert GB to bytes
    resource.setrlimit(resource.RLIMIT_AS, (soft, hard))

In [4]:
def parse_knapsack_csv(file_path):
    instances = []
    with open(file_path, 'r') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            weights = list(map(float, row['Weights'].strip('[]').split(",")))
            prices = list(map(float, row['Prices'].strip('[]').split(",")))
            capacity = float(row['knapsack_capacity'])
            items = [(i + 1, prices[i], weights[i]) for i in range(len(weights))]
            instances.append((len(items), items, weights, prices, capacity))
    return instances

In [ ]:
def solve_knapsack(items, capacity, ram):
    set_memory_limit_gb(ram)
    m = Model("min_knapsack")
    m.setParam('OutputFlag', 1)
    m.setParam("MIPGap", 0)
    m.setParam('Heuristics', 1)
    m.setParam('Presolve', 1)
    m.setParam('TimeLimit', 300)

    num_items = len(items)
    profit = [item[1] for item in items]
    weight = [item[2] for item in items]

    x = m.addVars(num_items, vtype=GRB.BINARY, name="x")

    #  Minimize total profit 
    m.setObjective(quicksum(profit[i] * x[i] for i in range(num_items)), GRB.MINIMIZE)

    #  total weight must be at least the required capacity 
    m.addConstr(quicksum(weight[i] * x[i] for i in range(num_items)) >= capacity, "weight_requirement")

    start_time = time.time()
    try:
        mem_before = get_memory_usage_in_kb()
        m.optimize()
        mem_after = get_memory_usage_in_kb()
        end_time = time.time()

        iter_count = m.getAttr(GRB.Attr.IterCount)
        node_count = m.getAttr(GRB.Attr.NodeCount)
        sol_count  = m.getAttr(GRB.Attr.SolCount)

        time_taken = end_time - start_time
        peak_memory = max(mem_before, mem_after)

        result = {
            "objective_value": None, "selected_items": [], "time_taken": time_taken,
            "peak_memory": peak_memory, "iterations": None, "nodes": None, "solutions": None
        }

        if m.status == GRB.OPTIMAL:
            result["objective_value"] = m.objVal
            result["selected_items"] = [items[i][0] for i in range(num_items) if x[i].x > 0.5]
            result["iterations"] = iter_count
            result["nodes"] = node_count
            result["solutions"] = sol_count
        else:
            result.update({k: np.nan for k in result})
            result["peak_memory"] = peak_memory

    except Exception as e:
        print(f"Optimization failed: {e}")
        result = {key: np.nan for key in ["objective_value", "selected_items", "time_taken", "peak_memory", "iterations", "nodes", "solutions"]}

    return result

In [ ]:
def process_knapsack_instances(input_csv_path, output_csv_path):
    # Parse the knapsack dataset
    instances = parse_knapsack_csv(input_csv_path)
    results = []
    ram = 256
    cpu_cores = 32
    
    
    for instance_idx, (n, items, weights, prices, capacity) in tqdm(enumerate(instances, start=1), total=len(instances), desc="Processing instances"):
        output = solve_knapsack(items, capacity, ram)
        result = {
            'number_of_elements': n,
            'capacity': capacity,
            'total_profit': output["objective_value"],
            'solution_time': output["time_taken"],
            'ram': ram,
            'cpu_cores': cpu_cores,
            'peak_memory': output["peak_memory"]
        }
        results.append(result)
 
   
    fieldnames = list(results[0].keys())
    
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(merged_results)
    
    print(f"Rsults saved {output_csv_path}")

In [ ]:
base_directory = "dataset_200.csv" #Specify the input dataset filepath
output_csv = "output.csv" #Specify the output path
process_knapsack_instances(base_directory, output_csv)

Processing instances:   0%|          | 0/199 [00:00<?, ?it/s]

Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0
Set parameter Heuristics to value 1
Set parameter Presolve to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD EPYC 7H12 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  300
MIPGap  0
Heuristics  1
Presolve  1

Optimize a model with 1 rows, 400 columns and 400 nonzeros
Model fingerprint: 0xd8af85e4
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+07]
  Objective range  [3e+00, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+08, 1e+08]
Found heuristic solution: objective 9.773852e+07
Presolve time: 0.00s
Presolved: 1 rows, 400 columns, 400 nonzeros
Variable types: 0 c

Processing instances:   1%|          | 1/199 [00:00<00:22,  8.95it/s]

Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0
Set parameter Heuristics to value 1
Set parameter Presolve to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD EPYC 7H12 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  300
MIPGap  0
Heuristics  1
Presolve  1

Optimize a model with 1 rows, 800 columns and 800 nonzeros
Model fingerprint: 0x8f2d24ad
Variable types: 0 continuous, 800 integer (800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+07]
  Objective range  [2e+00, 7e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+07, 9e+07]
Found heuristic solution: objective 9.916115e+07
Presolve time: 0.00s
Presolved: 1 rows, 800 columns, 800 nonzeros
Variable types: 0 continuous, 800 integer (800 binary)

Root relaxation: objective 5.7131

Processing instances:   2%|▏         | 3/199 [00:00<00:13, 14.37it/s]

...

Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0
Set parameter Heuristics to value 1
Set parameter Presolve to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD EPYC 7H12 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  300
MIPGap  0
Heuristics  1
Presolve  1

Optimize a model with 1 rows, 970 columns and 970 nonzeros
Model fingerprint: 0x8d50be72
Variable types: 0 continuous, 970 integer (970 binary)
Coefficient statistics:
  Matrix range     [4e+04, 1e+08]
  Objective range  [1e+04, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+07, 3e+07]
Found heuristic solution: objective 9.499210e+07
Presolve removed 0 rows and 321 columns
Presolve time: 0.01s
Presolved: 1 rows, 649 columns, 649 nonzeros
Variable types: 0 continuous, 649 integer (649 bi

Processing instances:  99%|█████████▉| 198/199 [00:12<00:00, 18.51it/s]

Set parameter OutputFlag to value 1
Set parameter MIPGap to value 0
Set parameter Heuristics to value 1
Set parameter Presolve to value 1
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: AMD EPYC 7H12 64-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 128 physical cores, 256 logical processors, using up to 32 threads

Non-default parameters:
TimeLimit  300
MIPGap  0
Heuristics  1
Presolve  1

Optimize a model with 1 rows, 400 columns and 400 nonzeros
Model fingerprint: 0x74c96879
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [2e+00, 6e+09]
  Objective range  [1e+00, 6e+09]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+09, 7e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 6.797247e+09
Presolve time: 0.00s
Presolved: 1 rows, 400 col

Processing instances: 100%|██████████| 199/199 [00:13<00:00, 15.26it/s]

Merged results saved to ./results_cpp/final_min/gurobi/gurobi_32_256.csv
